# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   803.554816 -0.491638  1234  803.063178  804.046455  1606.126356   
6   625.929413  0.629283  1234  626.558696  625.300130  1253.117391   
7   684.101982  0.283373  1234  684.385356  683.818609  1368.770712   
8     3.897689  0.627432  1234    4.525121    3.270257     9.050242   
9   477.115934 -0.800970  1234  476.314965  477.916904   952.629929   
10   45.293285  0.926330  1234   46.219615   44.366955    92.439230   
11  708.919774 -0.738897  1234  708.180876  709.658671  1416.361752   
12  668.944002 -0.442189  1234  668.501813  669.386191  1337.003627   
13  480.793091 -0.435247  1234  480.357844  481.228338   960.715688   
14  226.815163  0.196367  1234  227.011530  226.618796   454.023061   

              f  
5   1608.092910  
6   1250.600260  
7   1367.637218  
8      6.540515  
9    955.833807  
10    88.733910  
11  1419.317342  
12  1338.772383  
13   962.456676  
14   453.237591

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -803.063178   804.046455
6   -626.558696   625.300130
7   -684.385356   683.818609
8     -4.525121     3.270257
9   -476.314965   477.916904
10   -46.219615    44.366955
11  -708.180876   709.658671
12  -668.501813   669.386191
13  -480.357844   481.228338
14  -227.011530   226.618796

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -803.063178   804.046455
6   -626.558696   625.300130
7   -684.385356   683.818609
8     -4.525121     3.270257
9   -476.314965   477.916904
10   -46.219615    44.366955
11  -708.180876   709.658671
12  -668.501813   669.386191
13  -480.357844   481.228338
14  -227.011530   226.618796

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-803.06317818,  804.04645477],
       [-626.55869572,  625.30013003],
       [-684.38535595,  683.81860903],
       [  -4.52512109,    3.2702574 ],
       [-476.31496453,  477.91690353],
       [ -46.21961477,   44.36695478],
       [-708.18087622,  709.65867116],
       [-668.50181339,  669.38619131],
       [-480.35784384,  481.22833784],
       [-227.0115303 ,  226.61879555]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1604.126356  -1608.092910
6    1251.117391  -1250.600260
7    1366.770712  -1367.637218
8       7.050242     -6.540515
9     950.629929   -955.833807
10     90.439230    -88.733910
11   1414.361752  -1419.317342
12   1335.003627  -1338.772383
13    958.715688   -962.456676
14    452.023061   -453.237591

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf